In [1]:
import os
import joblib
import pandas as pd
import numpy as np
import time
from scipy.sparse import load_npz
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Modeller
from sklearn.feature_selection import SelectPercentile, f_classif 
from sklearn.svm import LinearSVC 

# --- KLASÖR AYARLARI ---
BASE_DIR = r"C:\Users\goktu\workspace\datacops"        
OUTPUT_DIR = os.path.join(BASE_DIR, "model_goktug")   

# Klasörün var olduğundan emin olalım (Yoksa oluşturur)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Okuma Yeri: {BASE_DIR}")
print(f"Kaydetme Yeri: {OUTPUT_DIR}")
print("Ayarlar tamamlandı. ")

Okuma Yeri: C:\Users\goktu\workspace\datacops
Kaydetme Yeri: C:\Users\goktu\workspace\datacops\model_goktug
Ayarlar tamamlandı. 


In [2]:
print("Veriler yükleniyor...")
X_train_full = load_npz(os.path.join(BASE_DIR, "X_train_ready.npz"))
X_test_ready = load_npz(os.path.join(BASE_DIR, "X_test_ready.npz"))
feature_names = joblib.load(os.path.join(BASE_DIR, "feature_names.pkl"))

df_train = pd.read_csv(os.path.join(BASE_DIR, "train_scaled.csv"))
df_test = pd.read_csv(os.path.join(BASE_DIR, "test_scaled.csv"))
y_train_full = df_train["Crm Cd"].astype(str)
y_test = df_test["Crm Cd"].astype(str)

print(f"Veri Hazır. Boyut: {X_train_full.shape}")

Veriler yükleniyor...
Veri Hazır. Boyut: (848024, 1786)


In [3]:
SAMPLE_SIZE = 500000 

print(f"Hız için {SAMPLE_SIZE} satırlık örneklem alınıyor...")

try:
    X_train_bireysel, _, y_train_bireysel, _ = train_test_split(
        X_train_full, y_train_full, train_size=SAMPLE_SIZE, 
        stratify=y_train_full, random_state=42
    )
except ValueError:
    print("! Stratify olmadı, rastgele alınıyor...")
    X_train_bireysel, _, y_train_bireysel, _ = train_test_split(
        X_train_full, y_train_full, train_size=SAMPLE_SIZE, random_state=42
    )

print("Örneklem hazır.")

Hız için 500000 satırlık örneklem alınıyor...
Örneklem hazır.


In [4]:
print("\n--- AŞAMA 1: Feature Selection (SelectPercentile) ---")
print("Özelliklerin en iyi %15'lik dilimi seçiliyor...")

# percentile=15: Özelliklerin en iyi %15'ini al
selector = SelectPercentile(score_func=f_classif, percentile=15)

# İstatistiği hesapla
selector.fit(X_train_bireysel, y_train_bireysel)

# Seçimi uygula
X_train_selected = selector.transform(X_train_bireysel)
X_test_selected = selector.transform(X_test_ready)

# Kaç özellik seçildiğini görelim
print(f"Seçim Tamamlandı ")
print(f"Başlangıç Özellik Sayısı: {X_train_bireysel.shape[1]}")
print(f"Seçilen Özellik Sayısı : {X_train_selected.shape[1]} (Toplamın %15'i)")


--- AŞAMA 1: Feature Selection (SelectPercentile) ---
Özelliklerin en iyi %15'lik dilimi seçiliyor...
Seçim Tamamlandı 
Başlangıç Özellik Sayısı: 1786
Seçilen Özellik Sayısı : 268 (Toplamın %15'i)


C:\Users\goktu\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\goktu\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [5]:
print("\n--- AŞAMA 2: Model Eğitimi (Linear SVM) ---")
print("Support Vector Machine eğitiliyor... (Geometrik Sınıflandırma)")

# LinearSVC: Büyük ve seyrek veriler için optimize edilmiş SVM
clf = LinearSVC(
    C=0.1,              # Düzenlileştirme (Düşük C = Daha basit sınırlar)
    dual=False,         # n_samples > n_features olduğu için False yapıyoruz (Hızlanır)
    max_iter=1000,
    random_state=42,
    verbose=1           # İlerlemeyi göster
)

start_time = time.time()
clf.fit(X_train_selected, y_train_bireysel)
end_time = time.time()

print(f"Model Eğitimi Tamamlandı ({end_time - start_time:.2f} saniye) ")


--- AŞAMA 2: Model Eğitimi (Linear SVM) ---
Support Vector Machine eğitiliyor... (Geometrik Sınıflandırma)
[LibLinear]Model Eğitimi Tamamlandı (1783.41 saniye) 


In [6]:
print("Test verisi üzerinde tahmin yapılıyor...")
y_pred = clf.predict(X_test_selected)

# Doğruluk Oranı
acc = accuracy_score(y_test, y_pred)
print(f"\n LINEAR SVM DOĞRULUK ORANI: {acc * 100:.2f}%")

# Submission Dosyası
submission_df = pd.DataFrame({
    "DR_NO": df_test["DR_NO"],
    "Crm Cd": y_pred
})

filename = "submission_goktug_svm.csv"

# --- DEĞİŞİKLİK BURADA: Dosyayı senin klasörüne kaydediyoruz ---
save_path = os.path.join(OUTPUT_DIR, filename) 
# -------------------------------------------------------------

submission_df.to_csv(save_path, index=False)

print(f" Dosya başarıyla oluşturuldu: {filename}")
print(f" Kaydedildiği Klasör: {OUTPUT_DIR}")

Test verisi üzerinde tahmin yapılıyor...

 LINEAR SVM DOĞRULUK ORANI: 29.23%
 Dosya başarıyla oluşturuldu: submission_goktug_svm.csv
 Kaydedildiği Klasör: C:\Users\goktu\workspace\datacops\model_goktug
